In [3]:
from tools.web_tool import WebTool
import json
import openai
from pydantic import BaseModel, Field
from typing import List
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create clientkey
client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key='',
)

In [4]:
class SubTask(BaseModel):
    sub_task : str = Field(description="A sub-task name to be done to complete the given task.")
    sub_task_description : str = Field(description="Detailed description about the subtask")

class ListOfSubTasks(BaseModel):
    sub_tasks : List[SubTask] = Field(description="List of subtask to be done to complete the given task at hand.")

In [5]:
user_query = "Hydrogen fule in automotive industry analysis"
message = [
    {"role": "system", "content": "You are a helpful and experienced market analyst. Use your own intelligence to provide the tasks and subtasks for completing the GOAL. Dont use any formatting, Answer in provided JSON schema"},
    {"role": "user", "content": f"Tell subtasks to complete the GOAL : {user_query}"}
]
response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=message,
    response_format={"type": "json_object", "schema": ListOfSubTasks.model_json_schema()},
    frequency_penalty = 0.1,
    temperature=0
)
print(response.choices[0].message.content)

 {
  "sub_tasks": [
    {
      "sub_task_description": "Research and gather data on the current state of hydrogen fuel technology in the automotive industry."
    ,
      "sub_task": "Hydrogen Fuel Technology Research"
    },
    {
      "sub_task_description": "Analyze the market size and growth potential of hydrogen fuel cell vehicles."
    ,
      "sub_task": "Market Size and Growth Analysis"
    },
    {
      "sub_task_description": "Identify key players in the hydrogen fuel cell vehicle market and analyze their market share, strategies, and performance."
    ,
      "sub_task": "Key Players and Market Share Analysis"
    },
    {
      "sub_task_description": "Examine the regulatory environment and government policies affecting the adoption of hydrogen fuel cell vehicles."
    ,
      "sub_task": "Regulatory Environment and Government Policies Analysis"
    },
    {
      "sub_task_description": "Evaluate the infrastructure for hydrogen fueling stations and its impact on the gro

In [6]:
def get_sub_tasks(message):
    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=message,
        response_format={"type": "json_object", "schema": ListOfSubTasks.model_json_schema()},
        frequency_penalty = 0.1,
        temperature=0
    )
    response = response.choices[0].message.content
    r = json.loads(response)
    sub_tasks = []
    sub_tasks_description = []
    
    for i in r['sub_tasks']:
        sub_tasks.append(i['sub_task'])
        sub_tasks_description.append(i['sub_task_description'])

    return sub_tasks,sub_tasks_description,response

In [7]:
import time

x = time.time()
user_query = "Smart wearables business analysis"
data_dict = {user_query : {}}
message = [
    {"role": "system", "content": "You are a helpful and experienced market analyst. Use your own intelligence to provide the subtasks for completing the TASK. Dont use any formatting, Answer in provided JSON schema"},
    {"role": "user", "content": f"Tell subtasks to complete the TASK : {user_query}"}
]

print(f"User Query : {user_query}\n\n")
sub_tasks,sub_task_descriptions,assistants = get_sub_tasks(message) 
    
for i in range(len(sub_tasks)):
    print(f"Running {i+1}/{len(sub_tasks)}")
    query = sub_task_descriptions[i]
    temp_msg = message.copy()
    temp_msg.append({"role":"assistant","content":assistants})
    temp_msg.append({"role":"user","content":f"Now Tell me subtasks to complete the TASK : {query}"})
    sts,stds,_ = get_sub_tasks(temp_msg)
    response = list(zip(sts,stds))
    data_dict[user_query][sub_tasks[i]] = response

print("time taken" , time.time()-x)

User Query : Smart wearables business analysis


Running 1/7
Running 2/7
Running 3/7
Running 4/7
Running 5/7
Running 6/7
Running 7/7
time taken 161.1968698501587


In [8]:
import json
with open("pizza_example.json",'w') as file:
    json.dump(data_dict,file,indent=4)

In [26]:
from tools.json2html_report import json2html_report

In [27]:
import json
with open("pizza_example.json",'r') as file:
    d = json.load(file)

In [28]:
r = {}

In [31]:
for i in d['Smart wearables business analysis']:
    da = []
    for j in d['Smart wearables business analysis'][i]:
        da.append({'Sub Task':j[0].strip(),'Description':j[1].strip()})
    r[i] = da

In [ ]:
print(json2html_report(r,'Pizza cafe business analysis'))

In [1]:
data = {
  "checked": [
    "Identify and gather data from reputable sources on the current market size of the smart wearables industry, including revenue and unit sales.",
    "Identify potential challenges to market growth, such as regulatory issues or competition from other industries.",
    "Create a list of key players in the smart wearables market through research and data collection.",
    "Analyze the product offerings of each key player, including features, pricing, and target markets.",
    "Gather data on demographic trends in the smart wearables market, including age, gender, income level, and geographic location.",
    "Investigate consumer attitudes towards data privacy and security in the context of smart wearables devices.",
    "Research and identify recent technological advancements in the smart wearables industry, such as new materials, sensors, and battery technologies.",
    "Explore potential use cases for new product innovations and emerging technologies in the smart wearables industry.",
    "Research and analyze existing regulations and policies that affect the smart wearables industry in major markets.",
    "Identify key competitors in the smart wearables market and analyze their strengths, weaknesses, opportunities, and threats (SWOT analysis).",
    "Summarize key findings from previous sub-tasks related to market size, growth rate, key players, consumer trends, technological advancements, and opportunities/challenges.",
    "Evaluate regulatory requirements and potential risks associated with entering or expanding in the smart wearables market."
  ],
  "expanded": [
    "Market_Size_and_Growth_Rate",
    "Key_Player_Analysis",
    "Consumer_Trends",
    "Technological_Impact",
    "Opportunities_and_Challenges",
    "Recommendations"
  ]
}